In [27]:
import os
import sys
import torch
import torch.nn as nn
import accimage
from PIL import Image
from imageio import imread
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms, set_image_backend, get_image_backend
import data_utils
import numpy as np
import pandas as pd
import pickle
import torch.nn.functional as F

%reload_ext autoreload
%autoreload 2

# https://github.com/pytorch/accimage
set_image_backend('accimage')
get_image_backend()

'accimage'

In [2]:
root_dir = '/n/mounted-data-drive/COAD/'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.ToTensor()
msi_path = '/home/sxchao/MSI_prediction/tcga_project/msi_raw_data.xlsx'
msi_raw = pd.read_excel(msi_path)

In [5]:
cut_off = 50
msi_raw.rename(columns={'Tumor type':'tumor_type', 
                        'Donor id':"donor_id", 
                        'Tumor sample id':'tumor_id',
                        'Normal sample id':'normal_id',
                        'Number of mutations A motif':'muts_A', 
                        'Number of covered A loci':'covg_A',
                        'Number of mutations C motif':'muts_C', 
                        'Number of covered C loci':'covg_C',
                        'Number of mutations AC motif':'muts_AC', 
                        'Number of covered AC loci':'covg_AC',
                        'Number of mutations AG motif':'muts_AG', 
                        'Number of covered AG loci':'covg_AG'}, 
               inplace=True)

msi_raw['muts_tot'] = msi_raw['muts_A'] + msi_raw['muts_C'] + msi_raw['muts_AC'] + msi_raw['muts_AG']
msi_raw['msi'] = msi_raw['muts_tot'] > cut_off
msi_raw.msi = msi_raw.msi.astype(int)
#msi_raw.tail(3)

In [6]:
coad_msi = msi_raw.loc[msi_raw['tumor_type']=='COAD','donor_id'].values


In [7]:
sample_name = coad_msi[-1]
name_len = len(sample_name)
#sample_name

In [8]:
coad_full_name = os.listdir(root_dir)
coad_img = np.array([v[0:name_len] for v in coad_full_name])
#len(coad_img), coad_img[5], coad_full_name[5]

In [9]:
coad_both = np.intersect1d(coad_img, coad_msi)

In [10]:
sample_names = []
for sample in coad_both:
    if sample != 'TCGA-A6-2675': # 5.0 empty for 'TCGA-A6-2675-01Z-00-DX1.d37847d6-c17f-44b9-b90a-84cd1946c8ab'
        key = np.argwhere(coad_img == sample).squeeze()
        if key.size != 0:
            sample_names.append(coad_full_name[key][:-4])

In [11]:
msi_raw.set_index('donor_id', inplace=True)
#msi_raw.head(3)

In [12]:
np.random.seed(seed=29)
reorder = np.random.permutation(len(sample_names))
train = reorder[:int(np.floor(len(sample_names)*0.8))]
val = reorder[int(np.floor(len(sample_names)*0.8)):]
len(train), len(val)

(196, 50)

In [14]:
sample_annotations = {}
sample_names = np.array(sample_names)
for sample_name in sample_names[train]:
    sample_annotations[sample_name] = msi_raw.loc[sample_name[0:name_len], 'msi']

In [67]:
train_set = data_utils.TCGADataset_tiles(sample_annotations, root_dir, transform=transform)


In [82]:
np.sum([len(j) for j in train_set.jpegs][0:4])

3525